# Exercise 2

In [ ]:
# Nice, "write the metropolis algorithm"
import metropolis
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Animation to verify
%matplotlib qt5
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider
animation_system = metropolis.Metropolis2D((100, 100), 1, 0)
animation_T = 1.5

fig, (ax2, ax1) = plt.subplots(2,1)
ax1.set_xticks([])
ax1.set_yticks([])
im1 = ax1.matshow(animation_system.spins)
# ax2.spines["top"].set_visible(True)
# ax2.spines["right"].set_visible(True)
T_slider = Slider(ax2, 'Temperature ', valmin=0, valmax=5, 
             valinit=1, valfmt='%.2f K/k_B', facecolor='#cc7000')
# fig.tight_layout()

def animation(_):
    for _ in range(100 * 100):
        animation_system.perform_iter(animation_T)
    im1.set_data(animation_system.spins)
    return im1,

def change_T(_):
    global animation_T
    animation_T = T_slider.val
T_slider.on_changed(change_T)

ani = FuncAnimation(fig, animation, blit=False, interval=50)
plt.show()